In [26]:
import warnings
warnings.filterwarnings('ignore')

In [27]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from keras.constraints import MaxNorm

In [28]:
# Epoch y Batch Optimization
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(32, input_shape=(5,), activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# fix random seed for reproducibility
seed = 7
tf.random.set_seed(seed)
# load dataset
dataset = pd.read_excel("default_cc_clients.xlsx", header=1)
# split into input (X) and output (Y) variables
dataset.drop(['ID', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6'], axis=1, inplace=True)
dataset.drop(['BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6'], axis=1, inplace=True)
dataset.drop(['PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6'], axis=1, inplace=True)

In [29]:
dataset.head()
X = dataset.iloc[:,0:5]
Y = dataset.iloc[:,5]

In [30]:
X

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE
0,20000,2,2,1,24
1,120000,2,2,2,26
2,90000,2,2,2,34
3,50000,2,2,1,37
4,50000,1,2,1,57
...,...,...,...,...,...
29995,220000,1,3,1,39
29996,150000,1,3,2,43
29997,30000,1,2,2,37
29998,80000,1,3,1,41


In [31]:
Y

0        1
1        1
2        0
3        0
4        0
        ..
29995    0
29996    0
29997    1
29998    1
29999    1
Name: default payment next month, Length: 30000, dtype: int64

In [32]:

# create model
model = KerasClassifier(model=create_model, verbose=0)

# define the grid search parameters
batch_size = [60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)

In [33]:
# Grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=3)
grid_result = grid.fit(X, Y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.775000 using {'batch_size': 60, 'epochs': 100}
0.592100 (0.262266) with: {'batch_size': 60, 'epochs': 10}
0.592900 (0.262832) with: {'batch_size': 60, 'epochs': 50}
0.775000 (0.005374) with: {'batch_size': 60, 'epochs': 100}
0.407067 (0.262855) with: {'batch_size': 80, 'epochs': 10}
0.592333 (0.262431) with: {'batch_size': 80, 'epochs': 50}
0.589400 (0.260395) with: {'batch_size': 80, 'epochs': 100}
0.592467 (0.262526) with: {'batch_size': 100, 'epochs': 10}
0.592767 (0.262738) with: {'batch_size': 100, 'epochs': 50}
0.769000 (0.012737) with: {'batch_size': 100, 'epochs': 100}


In [34]:
# Random search
grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_jobs=1, cv=3)
grid_result = grid.fit(X, Y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.767700 using {'epochs': 100, 'batch_size': 80}
0.592600 (0.262620) with: {'epochs': 10, 'batch_size': 60}
0.577167 (0.252056) with: {'epochs': 50, 'batch_size': 60}
0.693100 (0.063185) with: {'epochs': 100, 'batch_size': 60}
0.593133 (0.262997) with: {'epochs': 10, 'batch_size': 80}
0.725400 (0.039918) with: {'epochs': 50, 'batch_size': 80}
0.767700 (0.014171) with: {'epochs': 100, 'batch_size': 80}
0.591100 (0.261568) with: {'epochs': 10, 'batch_size': 100}
0.568267 (0.245739) with: {'epochs': 50, 'batch_size': 100}
0.565200 (0.245205) with: {'epochs': 100, 'batch_size': 100}


## OPtimization Alg. Tunning

In [39]:
# Optimization Alg. Tunning
def create_model():
 # create model
 model = Sequential()
 model.add(Dense(12, input_shape=(5,), activation='relu'))
 model.add(Dense(1, activation='sigmoid'))
 # return model without compile
 return model

In [40]:
# fix random seed for reproducibility
seed = 7
tf.random.set_seed(seed)
# create model
model = KerasClassifier(model=create_model, loss="binary_crossentropy", epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)

In [41]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=3)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

## Learning Rate Opt.

In [ ]:

def create_model():
 # create model
 model = Sequential()
 model.add(Dense(12, input_shape=(5,), activation='relu'))
 model.add(Dense(1, activation='sigmoid'))
 return model

In [ ]:
# fix random seed for reproducibility
seed = 7
tf.random.set_seed(seed)

# create model
model = KerasClassifier(model=create_model, loss="binary_crossentropy", optimizer="SGD", epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1]
momentum = [0.2, 0.4, 0.6]
param_grid = dict(optimizer__learning_rate=learn_rate, optimizer__momentum=momentum)

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=3)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

## Activation Function Opt.

In [ ]:
def create_model(activation='relu'):
 # create model
 model = Sequential()
 model.add(Dense(12, input_shape=(5,), kernel_initializer='uniform', activation=activation))
 model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
 # Compile model
 model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
 return model


In [ ]:
# fix random seed for reproducibility
seed = 7
tf.random.set_seed(seed)

# create model
model = KerasClassifier(model=create_model, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
param_grid = dict(model__activation=activation)

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=3)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

## Number of neurons

In [ ]:
def create_model(neurons):
 # create model
 model = Sequential()
 model.add(Dense(neurons, input_shape=(5,), kernel_initializer='uniform', activation='linear', kernel_constraint=MaxNorm(4)))
 #model.add(Dropout(0.2))
 model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
 # Compile model
 model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
 return model

In [ ]:
# fix random seed for reproducibility
seed = 7
tf.random.set_seed(seed)

# create model
model = KerasClassifier(model=create_model, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
neurons = [1, 5, 10, 15, 20, 25, 30]
param_grid = dict(model__neurons=neurons)


In [ ]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=3)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))